In [53]:
import numpy as np
import pandas as pd
import math as m
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
# Testing..attention please..nuisance, you sent, who sent for me?

In [54]:
# defininf the states
# defininsg coefficeients sybols

def Creating_states( coeff = [1/(2**.5),1/(2**.5),1,0] , Abstract = False):    # coeff list like [a0,a1,b0,b1]
    if Abstract == True:
        a0 = sym.symbols('a0')
        a1 = sym.symbols('a1')
        b0 = sym.symbols('b0')
        b1 = sym.symbols('b1')
    else:
        a0 = coeff[0]
        a1 = coeff[1]
        b0 = coeff[2]
        b1 = coeff[3]
    psi0 = [a0,a1]      # defining states
    psi1 = [b0,b1]
    return([psi0,psi1])

In [55]:
psi0 = Creating_states(Abstract=False)[0]
psi1 = Creating_states(Abstract=False)[1]

psi0

[0.7071067811865475, 0.7071067811865475]

In [56]:
psi0sq = []
psi1sq = []
[[psi0sq.append(i*j) for i in psi0] for j in psi0]
[[psi1sq.append(i*j) for i in psi1] for j in psi1]   # creating list of coeff for squared states (intermeditate step)

psi0psi0 = [psi0sq[0], (psi0sq[1]**2+psi0sq[2]**2)**(.5), psi0sq[3]]        # the second element is the normalized coefficient due to the addition of states (01+10). See OneNote writeup for details
psi1psi1 = [psi1sq[0], (psi1sq[1]**(2)+psi1sq[2]**(2))**(.5), psi1sq[3]]            # symmetric states in basis (|00>, normalized version of (|01>+|10>),|11>)

# psi0psi0
# sum([i**2 for i in psi0psi0])   # adding to 1 now.

In [57]:
#creating the density matrix rho
# rho is 1/2*(|psi0psi0Xpsi0psi0|+|psi1psi1Xpsi1psi1|)
# findiing the conjugate vectors 
vec_psi0psi0 = sym.Matrix(psi0psi0)     # convertying to sympy vectors for easy matrix multiplication
vec_psi1psi1 = sym.Matrix(psi1psi1)

rho = (1/2)*(vec_psi0psi0*vec_psi0psi0.T  + vec_psi1psi1*vec_psi1psi1.T)       # density matrix with priors 1/2 each.
rho

Matrix([
[            0.625, 0.176776695296637,             0.125],
[0.176776695296637,              0.25, 0.176776695296637],
[            0.125, 0.176776695296637,             0.125]])

In [58]:
# creating the SIC POVM matrices
w = m.e**((2/3)*m.pi*(1j))
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]
# sum([np.trace(POVM_elts[i]) for i in range(9)])`# trace sum checks out`

In [59]:
# prob_vec = [(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] #[(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
# prob_vec
sum(prob_vec)        # prob not adding to 1?
# print(prob_vec)


0.999999999999999

In [60]:
# # verifying that the trace still holds to be the same, equalt to .111 for this weird POVM element.
# c = .00308641975308642*2.71828182845905**(4.18879020478639*(sym.I))

# p = cm.polar(c)
# cm.rect(p[0], p[1])

# r = .00617-.0015
# i = .0026

# ab = (r**2+i**2)**(.5)
# 2*ab # verified

In [61]:
N = 10000       # number of measurements  

POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']       # symbols to indicate collapsed direction
#prob distribution is simply the corresponding elements of the prob_vec
collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]         # working babieee
pj_num_vec = [i/N for i in nj_vec]         # working babieee
([i*N for i in prob_vec] , nj_vec)   # comparing the theoretical and experimental collapse prob.

([35.7443490112103,
  833.333333333333,
  869.077682344544,
  919.627825494394,
  1458.33333333333,
  1752.96115882773,
  919.627825494394,
  1458.33333333333,
  1752.96115882773],
 [32, 840, 815, 951, 1474, 1755, 915, 1445, 1773])

In [62]:
M = [[np.trace(np.dot(POVM_elts[i],POVM_elts[j])) for i in range(len(POVM_elts))] for j in range(len(POVM_elts))]     # creating M matrix using POVM definition

In [63]:
u_0 = [1/3 for i in range(9)]           # cerating u_0 vector, to create the inverse matrix
M_inv = 3*np.outer(u_0,u_0) + 12*(np.eye(9) - np.outer(u_0,u_0))        # creating the inverse matrix
# np.trace(np.dot(M,M_inv))       # gives identity, checks out
r_vec = np.dot(M_inv,pj_num_vec)


In [64]:
rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]
[i for i in rho_num_list]

[array([[-0.        +0.j, -0.        +0.j, -0.        +0.j],
        [-0.        +0.j, -0.16026667+0.j,  0.16026667-0.j],
        [-0.        +0.j,  0.16026667+0.j, -0.16026667+0.j]]),
 array([[ 0.00133333+0.j,  0.        +0.j, -0.00133333-0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j],
        [-0.00133333+0.j,  0.        +0.j,  0.00133333+0.j]]),
 array([[-0.00366667+0.j,  0.00366667-0.j, -0.        +0.j],
        [ 0.00366667+0.j, -0.00366667+0.j, -0.        +0.j],
        [-0.        +0.j, -0.        +0.j, -0.        +0.j]]),
 array([[0.        +0.00000000e+00j, 0.        +0.00000000e+00j,
         0.        +0.00000000e+00j],
        [0.        +0.00000000e+00j, 0.02353333+4.95988426e-19j,
         0.01176667-2.03804645e-02j],
        [0.        +0.00000000e+00j, 0.01176667+2.03804645e-02j,
         0.02353333-1.10223218e-19j]]),
 array([[ 0.12813333+0.00000000e+00j,  0.        +0.00000000e+00j,
          0.06406667+1.10966722e-01j],
        [ 0.        +0.00000

In [65]:
rho_num = np.zeros_like(rho_num_list[0])

# Loop over the matrices and sum them element-wise
for matrix in rho_num_list:
    rho_num = np.add(rho_num, matrix)

rho_num 

array([[0.6204+0.00000000e+00j, 0.1898+3.11769145e-03j,
        0.1239+5.02294734e-03j],
       [0.1898-3.11769145e-03j, 0.2482+3.42427188e-18j,
        0.1802-6.23538291e-03j],
       [0.1239-5.02294734e-03j, 0.1802+6.23538291e-03j,
        0.1314+2.21217594e-18j]])

In [75]:
rho 
del_rho = rho_num-rho
del_rho
rho

Matrix([
[            0.625, 0.176776695296637,             0.125],
[0.176776695296637,              0.25, 0.176776695296637],
[            0.125, 0.176776695296637,             0.125]])

In [67]:
# evects = del_rho.eigenvects()
eigensystem = del_rho.eigenvects()

#separating eigenvalues of del rho
evects_0 = []
evals_0 = []

for i in range(3):
    evals_0.append(eigensystem[i][0])
    evects_0.append(eigensystem[i][-1])

evals = [sym.re(i) for i in evals_0]

norm_del_rho = max([abs(i) for i in evals])     # picking out the largest evalue



In [74]:
#eigenvalues of rho_num
eigensystem_num = sym.Matrix(rho_num).eigenvects()
evects_num = []
evals_num = []

for i in range(3):
    evals_num.append(eigensystem_num[i][0])
    evects_num.append(eigensystem_num[i][-1])

# evals = [sym.re(i) for i in evals_0]
evals_num = [sym.re(i) for i in evals_num]

zero_eval_rho_num = min(evals_num)                          # picking the minimum evalue 
zero_eval_index = evals_num.index(zero_eval_rho_num)        # finding index so we can pick the corresponding evector
perp_evect_rho_num = evects_num[zero_eval_index]
perp_evect_rho_num




[Matrix([
 [-0.0238170279137322 + 0.0119827494861015*I],
 [  0.599123558125629 - 0.0126744821682508*I],
 [ -0.799774442741972 - 0.0232448539150386*I]])]